## Libraries 

In [1]:
import pandas as pd
import numpy as np

import glob
import xarray as xr

import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing 

from sklearn.linear_model import LinearRegression

from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.utils.plotting import plot_series

import seaborn as sns

from standard_precip.spi import SPI
from standard_precip.utils import plot_index

In [2]:
#load df
feature_df = pd.read_csv('../data/feature_df.csv', header=0)
feature_df.index = range(1971, 2023)
feature_df.head(10)

,Canola_detrend,spi_4,spi_5,spi_6,spi_7,spi_8,spi_9,spi_10,avg_max_temp_4,avg_max_temp_5,avg_max_temp_6,avg_max_temp_7,avg_max_temp_8,avg_max_temp_9,avg_max_temp_10
1971,0.835685,1.552420,0.398286,1.608689,-1.138548,-0.619148,0.699883,1.150625,284.674011,291.586487,295.451691,297.247833,301.780426,292.659363,285.051239
1972,1.019651,0.310719,0.272510,-0.248256,-0.270669,-0.278103,-0.179716,-0.901951,282.146759,292.675629,296.554260,296.034912,299.861603,291.575226,282.966553
1973,3.180447,0.277076,-0.334090,0.603571,0.403538,0.782308,2.709394,0.711722,283.948242,291.120361,296.939117,298.817383,301.038696,291.306488,287.401154
1974,-0.681926,0.740812,1.373921,-1.551774,-0.435588,0.232750,-1.372053,-1.695853,281.231293,287.215179,296.320404,301.303864,296.313019,291.511688,288.256470
1975,-1.567468,2.002202,-0.518458,1.556986,0.509661,1.773032,0.895744,1.033422,275.753235,289.641205,294.908081,300.886871,296.023590,291.019562,285.494232
1976,0.523820,0.532937,-1.697363,0.961862,-1.446509,0.499695,-1.474134,-2.029217,283.979858,292.507629,296.083862,299.666107,300.495178,297.305878,283.504913
1977,0.591938,-1.059830,0.273647,-0.688714,-0.430323,-0.897585,1.339607,-1.142240,290.806793,297.221161,296.493164,300.909698,296.245300,290.973480,287.136871
1978,0.636888,-0.105057,0.681106,-1.141082,-0.218658,-0.842599,1.921957,-0.756340,283.612152,293.872833,297.419464,299.356598,300.507751,295.736267,287.023193
1979,-2.341333,0.840549,0.646930,-0.982525,0.189466,-1.968555,0.959279,-0.739282,276.046600,286.521606,296.374207,300.664185,297.919647,294.441315,284.963867
1980,-10.342722,-1.357788,-2.438554,-0.763792,-0.754718,2.122007,0.205853,0.806743,289.224030,296.703491,297.945221,301.438324,295.128296,292.048950,283.809509


## Weather data 

In [3]:
# load raw weather data with t2m, tp 
weather_raw = pd.read_csv('C:/Users/maris/python_notebooks/XAI_TS_Forecasting/notebooks/dist1_t2m_tp.csv', header=0)
weather_raw['time'] = pd.to_datetime(weather_raw['time'])
weather_raw = weather_raw.set_index('time')

# check for nan
contains_nan = weather_raw.isna().any().any()

weather_raw.head()

,t2m,tp,Canola
time,,,
1971-04-01 00:00:00,267.04202,0.010006,18.0
1971-04-01 01:00:00,266.61612,0.000009,18.0
1971-04-01 02:00:00,266.40143,0.000017,18.0
1971-04-01 03:00:00,266.30045,0.000024,18.0
1971-04-01 04:00:00,266.17166,0.000031,18.0


In [4]:
# function to calculate the longest consecutive true streak

def longest_consecutive_true_streak(series):
    
    # Convert the series to integers (True to 1, False to 0) for easier streak calculation
    as_ints = series.astype(int)
    # Calculate the difference to identify changes in streaks
    diff = as_ints.diff()
    # Start a new group every time there's a change from 0 to 1 (start of a new streak)
    groups = (diff == 1).cumsum()
    # Use the groups to isolate consecutive trues, then count them, keeping the max
    streak_lengths = as_ints.groupby(groups).sum()
    # Return the length of the longest streak
    return streak_lengths.max()

## Heat Wave Index 

In [5]:
# get max temp for one day 
dist1_df_hot = weather_raw 

# add extra columns containing the years and the months
dist1_df_hot['year'] = dist1_df_hot.index.year
dist1_df_hot['month'] = dist1_df_hot.index.month
dist1_df_hot['day'] = dist1_df_hot.index.day

# get variables year, month and canola from old data frame 
daily_df = weather_raw.resample('D').first()
daily_df = daily_df[["Canola","year","month"]]

# Resample the data to daily frequency and get the maximum temperature for each day
daily_max_temperature = dist1_df_hot['t2m'].resample('D').max()

#drop all nan ( values for month november-march)
daily_max_temperature = daily_max_temperature.dropna()

# divide in test and training data 
training_data = daily_max_temperature.loc[:'1989']
testing_data = daily_max_temperature.loc['1990':]

# calculate for every day the 90% quantile
quantile_90_series = training_data.groupby([training_data.index.month, training_data.index.day]).quantile(0.9)
quantile_90_series.index = quantile_90_series.index.map(lambda x: f"{x[0]:02d}-{x[1]:02d}")


test_df = testing_data.to_frame(name='value')
test_df['month_day'] = test_df.index.strftime('%m-%d')

# map the 90th percentile values from quantile_90_series to the test series
test_df['quantile_90'] = test_df['month_day'].apply(lambda x: quantile_90_series.get(x, pd.NA))

# compare each test value to its corresponding 90th percentile value
test_df['is_above_quantile_90'] = test_df['value'] > test_df['quantile_90']
test_df.drop(['month_day', 'quantile_90'], axis=1, inplace=True)

# Group the DataFrame by year, and apply the function to find the longest streak of True values
longest_streak_by_year = test_df.groupby(test_df.index.year)['is_above_quantile_90'].apply(longest_consecutive_true_streak)

#print(longest_streak_by_year)

## Summer days 

In [6]:
training_data = daily_max_temperature.loc[:'1989']
testing_data = daily_max_temperature.loc['1990':]

test_df_summer = testing_data.to_frame(name='t2m')
test_df_summer['month_day'] = test_df_summer.index.strftime('%m-%d')

# # map the 90th percentile values from quantile_90_series to the test series
# test_df['quantile_90'] = test_df['month_day'].apply(lambda x: quantile_90_series.get(x, pd.NA))

# compare each test value to its corresponding 90th percentile value
test_df_summer['above_25'] = test_df_summer['t2m'] > 298

test_df_summer["above_25"].iloc[100:120]

days_over_25 = test_df_summer.groupby(test_df_summer.index.year)['above_25'].apply(sum)

days_over_25

time
1990    79
1991    52
1992    40
1993    29
1994    40
1995    56
1996    57
1997    69
1998    55
1999    38
2000    40
2001    59
2002    59
2003    67
2004    28
2005    47
2006    72
2007    67
2008    48
2009    56
2010    46
2011    62
2012    73
2013    50
2014    47
2015    64
2016    55
2017    68
2018    73
2019    60
2020    81
2021    94
2022    71
Name: above_25, dtype: int64

## Cold Wave Index 

In [7]:
# Code from above with adjustments to the coldness

# get max temp for one day 
dist1_df_hot = weather_raw 

# add extra columns containing the years and the months
dist1_df_hot['year'] = dist1_df_hot.index.year
dist1_df_hot['month'] = dist1_df_hot.index.month
dist1_df_hot['day'] = dist1_df_hot.index.day

# get variables year, month and canola from old data frame 
daily_df = weather_raw.resample('D').first()
daily_df = daily_df[["Canola","year","month"]]

# Resample the data to daily frequency and get the maximum temperature for each day
daily_min_temperature = dist1_df_hot['t2m'].resample('D').min()

#drop all nan ( values for month november-march)
daily_min_temperature = daily_min_temperature.dropna()

# divide in test and training data 
training_data = daily_min_temperature.loc[:'1989']
testing_data = daily_min_temperature.loc['1990':]

# calculate for every day the 90% quantile
quantile_10_series = training_data.groupby([training_data.index.month, training_data.index.day]).quantile(0.1)
quantile_10_series.index = quantile_10_series.index.map(lambda x: f"{x[0]:02d}-{x[1]:02d}")


test_df_cold = testing_data.to_frame(name='value')
test_df_cold['month_day'] = test_df_cold.index.strftime('%m-%d')

# map the 90th percentile values from quantile_90_series to the test series
test_df_cold['quantile_10'] = test_df_cold['month_day'].apply(lambda x: quantile_10_series.get(x, pd.NA))

# compare each test value to its corresponding 90th percentile value
test_df_cold['is_under_quantile_10'] = test_df_cold['value'] < test_df_cold['quantile_10']
test_df_cold.drop(['month_day', 'quantile_10'], axis=1, inplace=True)


# Group the DataFrame by year, and apply the function to find the longest streak of True values
longest_streak_by_year_cold = test_df_cold.groupby(test_df.index.year)['is_under_quantile_10'].apply(longest_consecutive_true_streak)

#print(longest_streak_by_year_cold)

## Frost Days 

In [8]:
training_data = daily_max_temperature.loc[:'1989']
testing_data = daily_max_temperature.loc['1990':]

test_df_frost = testing_data.to_frame(name='t2m')
test_df_frost['month_day'] = test_df_frost.index.strftime('%m-%d')

# compare each test value to its corresponding 90th percentile value
test_df_frost['under_0'] = test_df_frost['t2m'] < 273

#test_df_frost["under_0"].iloc[100:120]

days_under_0 = test_df_frost.groupby(test_df_frost.index.year)['under_0'].apply(sum)

days_under_0

time
1990     0
1991     4
1992     3
1993     1
1994     0
1995     2
1996     8
1997     7
1998     0
1999     1
2000     3
2001     2
2002     8
2003     6
2004     1
2005     0
2006     1
2007     4
2008     0
2009     4
2010     1
2011     0
2012     2
2013    11
2014     6
2015     0
2016     0
2017     0
2018     9
2019     3
2020    11
2021     1
2022     6
Name: under_0, dtype: int64

## Longest Dry Spell 

In [9]:
dist1_df_perci = weather_raw.resample('D').sum()

# use data frame without zero to calculate the 5% quantile (only!)
dist1_df_perci_wo0 = dist1_df_perci[dist1_df_perci['tp'] != 0]

# Threshhold is 1mm or 0.001m, here precipitation is likely measured in m 

# find days with less than 0.001m rain 
dist1_df_perci_wo0['less_than_0.001'] = dist1_df_perci_wo0['tp'] < 0.001

# apply function 
longest_dry_spell_per_year = dist1_df_perci_wo0.groupby(dist1_df_perci_wo0.index.year)['less_than_0.001'].apply(longest_consecutive_true_streak)

#longest_dry_spell_per_year

C:\Users\maris\AppData\Local\Temp\ipykernel_52200\3952656548.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist1_df_perci_wo0['less_than_0.001'] = dist1_df_perci_wo0['tp'] < 0.001


## Longest Wet Spell

In [10]:
dist1_df_perci = weather_raw.resample('D').sum()

# use data frame without zero to calculate the 5% quantile (only!)
dist1_df_perci_wo0 = dist1_df_perci[dist1_df_perci['tp'] != 0]

# Threshhold is 1mm or 0.001m, here precipitation is likely measured in m 

# find days with less than 0.001m rain 
dist1_df_perci_wo0['more_than_0.001'] = dist1_df_perci_wo0['tp'] > 0.001

# apply function 
longest_wet_spell_per_year = dist1_df_perci_wo0.groupby(dist1_df_perci_wo0.index.year)['more_than_0.001'].apply(longest_consecutive_true_streak)

#longest_wet_spell_per_year

C:\Users\maris\AppData\Local\Temp\ipykernel_52200\3498330211.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist1_df_perci_wo0['more_than_0.001'] = dist1_df_perci_wo0['tp'] > 0.001


## 95th Quantile Precipitation 

In [11]:
training_data_pre = dist1_df_perci_wo0["tp"].loc[:'1989']
testing_data_pre = dist1_df_perci_wo0["tp"].loc['1990':]

# calculate for every day the 90% quantile
quantile_95_series = training_data_pre.groupby([training_data_pre.index.month, training_data_pre.index.day]).quantile(0.95)
quantile_95_series.index = quantile_95_series.index.map(lambda x: f"{x[0]:02d}-{x[1]:02d}")

test_df_pre = testing_data_pre.to_frame(name='tp')
test_df_pre['month_day'] = test_df_pre.index.strftime('%m-%d')

# map the 90th percentile values from quantile_90_series to the test series
test_df_pre['quantile_95'] = test_df_pre['month_day'].apply(lambda x: quantile_95_series.get(x, pd.NA))

# compare each test value to its corresponding 90th percentile value
test_df_pre['over_quantile_95'] = test_df_pre['tp'] > test_df_pre['quantile_95']
test_df_pre.drop(['month_day', 'quantile_95'], axis=1, inplace=True)

# Group the DataFrame by year, and apply the function to find the longest streak of True values
days_over_95 = test_df_pre.groupby(test_df_pre.index.year)['over_quantile_95'].apply(sum)

# days_over_95

In [ ]:
# average max/min temp for whole growing season 
# average min temp per month 